# Librerias

In [3]:
import pandas as pd
import re
import numpy as np
import pandas as pd

from sqlalchemy import create_engine, text, insert, Table, Column, Integer, String, MetaData, BIGINT

In [4]:
from warnings import filterwarnings
filterwarnings('ignore')

# Funciones globales

In [5]:
def nombre_estados(fila):
    return fila[:fila.find('!')]

In [6]:
name_columns = {
    'Label (Grouping)': 'name',
    'Arizona!!Estimate': 'Arizona',
    'Florida!!Estimate': 'Florida',
    'Georgia!!Estimate': 'Georgia', 
    'Illinois!!Estimate': 'Illinois', 
    'Michigan!!Estimate': 'Michigan',
    'North Carolina!!Estimate': 'North Carolina',
    'Ohio!!Estimate': 'Ohio',
    'Tennessee!!Estimate': 'Tennessee',
    'Texas!!Estimate': 'Texas',
    'Virginia!!Estimate': 'Virginia'
}

In [7]:
def uniones(tabla, etq):
    columnas = ["Arizona", "Florida", "Georgia", "Illinois", "Michigan", "North Carolina", "Ohio", "Tennessee", "Texas", "Virginia"] #Pensylvania, New York, California
    for col in columnas:
        tabla[f"{col}_x"] += tabla[f"{col}_y"]
    tabla = tabla.iloc[:, :11].assign(nombre=etq)
    return tabla

In [8]:
def separar(tabla, estado, etiq):
    tabla = tabla[["name",estado, "nombre"]]
    tabla = tabla.rename(columns={estado: "total"})
    tabla = tabla.assign(etiqueta=etiq)
    return tabla

In [9]:
def princ(tabla):
    state_mappings = [("Arizona_x", "Arizona"), ("Florida_x", "Florida"), ("Georgia_x", "Georgia"),
                      ("Illinois_x", "Illinois"), ("Michigan_x", "Michigan"), ("North Carolina_x", "North Carolina"),
                      ("Ohio_x", "Ohio"), ("Tennessee_x", "Tennessee"),
                      ("Texas_x", "Texas"), ("Virginia_x", "Virginia"), 
                      ]
    tables = []
    for state, state_code in state_mappings:
        table = separar(tabla, state, state_code)
        tables.append(table)
    tabla__ = pd.concat(tables)
    return tabla__

In [10]:
def limpiar_edad(fila):
    return fila.strip()

In [11]:
def general(etiq, one, two):
    tabla = pd.merge(one, two, on="name")
    tabla = uniones(tabla, etiq)
    tabla = princ(tabla)
    return tabla

In [12]:
def rangos(edades, inicial, porcentaje, etiqueta1, etiqueta2):
    rango_aum = edades[inicial:inicial+1]
    rango_aum["name"] = etiqueta1
    rango_aum2 = rango_aum.iloc[:,-10:] * porcentaje
    rango_aum2 = rango_aum2.astype(int)
    rango_aum.iloc[:,-10:] = rango_aum2
    edades = pd.concat([edades, rango_aum])

    rango_m = rango_aum.iloc[:,-10:] * (1-porcentaje)
    rango_m = rango_m.astype(int)
    rango_aum.iloc[:,-10:] = rango_m
    rango_aum["name"] = etiqueta2
    edades[inicial:inicial+1]= rango_aum
    return edades

In [13]:
def etnia(tabla, etiqu):
    
    tabla = tabla.rename(columns=name_columns)
    tabla.iloc[:,-10:] = tabla.iloc[:,-10:].apply(lambda x: x.str.replace(',', '').astype(int))
    tabla["name"] = tabla["name"].apply(limpiar_edad)
    tabla['name'] = tabla['name'].replace('18 and 19 years', '18 to 19 years')
    male_tabla = tabla[2:16]
    female_tabla = tabla[17:]

    tablas_etnias = [male_tabla, female_tabla]

    args_list = [(8, 0.65, "35 to 39 years", "40 to 44 years"), (9, 0.55, "45 to 49 years", "50 to 54 years"),
                (10, 0.55, "55 to 59 years", "60 to 64 years"), (11, 0.55, "65 to 69 years", "70 to 74 years"),
                (12, 0.55, "75 to 79 years", "80 to 44 years")]

    for t_etnia in range(len(tablas_etnias)):
        for args in args_list:
            tablas_etnias[t_etnia] = rangos(tablas_etnias[t_etnia], *args)

    tabla = general(etiqu, tablas_etnias[0], tablas_etnias[1])
    return tabla

In [14]:
def ampliar(tabla, n, rangos):
    for i in range(0,n):
        tabla = pd.concat([tabla, tabla])
    tabla = tabla[0:n] 
    tabla["name"] = rangos
    return tabla

In [15]:
def ultimos(tabla, porcentaje):
    tabla_2 = tabla["total"]
    tabla_2 = tabla_2 * porcentaje
    tabla_2 = tabla_2.astype(int)
    tabla["total"] = tabla_2
    return tabla

In [16]:
def ids_merge(tabla, estados, edades):
    tabla = tabla.rename(columns={
        "etiqueta":"estados", "name":"edad"
    })
    tabla1 = pd.merge(tabla, estados, how="left" ,on=["estados"])
    tabla1 = tabla1.rename(columns={
        "index":"estado"
    })
    tabla1 = pd.merge(tabla1, edades, how="left" ,on=["edad","estado"])
    return tabla1

In [62]:
def persona(one_r):
    one2= one_r[0:1]
    one2= ampliar(one2, 9, ["15 to 17 years", "18 to 19 years", "20 to 24 years", "25 to 29 years", 
                            "30 to 34 years", "35 to 39 years", "40 to 44 years", "45 to 49 years",
                            "50 to 54 years"])

    one3 = one_r[3:4]
    one3 = ampliar(one3, 3, ["75 to 79 years", "80 to 84 years", "85 years and over"])
    
    one4 = one_r[1:2]
    one4 = ampliar(one4, 2, ["55 to 59 years", "60 to 64 years"])

    one5 = one_r[2:3]
    one5 = ampliar(one5, 2, ["65 to 69 years", "70 to 74 years"])

    one_r = pd.concat([one2, one4])
    one_r = pd.concat([one_r, one5])
    one_r = pd.concat([one_r, one3])

    return one_r

In [130]:
def age_range_to_list(age_range_str):
    age_range = re.findall(r"\d+", age_range_str)
    age_range = [int(x) for x in age_range]
    start = age_range[0]
    if(len(age_range) > 1):
        end = age_range[1]
        age_list = [(i, i+4) for i in range(start, end+1, 5)]
    elif(len(age_range) == 1):
        if(start == 65):
            age_list = [(65,69),(70,74),(75,79),(80,84),(85)]
        elif(start == 25):
            age_list = [(5),(5,9),(10,14),(15,17),(18,19),(20,24)]
    return age_list

In [17]:
def codigo_a_estado(fila):
    if(fila == "TX"):
        fila = "Texas"
    elif(fila == "FL"):
        fila = "Florida"
    elif(fila == "NY"):
        fila = "New York"
    elif(fila == "PA"):
        fila = "Pennsylvania"
    elif(fila == "IL"):
        fila = "Illinois"
    elif(fila == "OH"):
        fila = "Ohio"
    elif(fila == "GA"):
        fila = "Georgia"
    elif(fila == "NC"):
        fila = "North Carolina"
    elif(fila == "MI"):
        fila = "Michigan"
    elif(fila == "CA"):
        fila = "California"
    elif(fila == "VA"):
        fila = "Virginia"
    elif(fila == "AZ"):
        fila = "Arizona"
    elif(fila == "TN"):
        fila = "Tennessee"
    return fila

In [18]:
state_filter = ["TX", "FL", "NY", "PA", "IL", "OH", "GA", "NC", "MI", "CA", "VA", "AZ", "TN"]

# Censo

## ESTADOS

In [19]:
poblacion = pd.read_csv("data_censo/poblacion.csv")

In [20]:
estados = pd.DataFrame(poblacion.iloc[1,1:11])
estados.iloc[:,-10:] = estados.iloc[:,-10:].apply(lambda x: x.str.replace(',', '').astype(int))
estados = estados.reset_index()
estado = []
for i in estados["index"].tolist():
     estado.append(i[:i.find('!')])
estados = pd.DataFrame({ "estados": estado,
                         "poblacion":estados[1].astype(int)
})
estados = estados.reset_index()
estados.rename(columns={"index":"id"})
estados

,index,estados,poblacion
0,0,Arizona,7276316
1,1,Florida,21781128
2,2,Georgia,10799566
3,3,Illinois,12671469
4,4,Michigan,10050811
5,5,North Carolina,10551162
6,6,Ohio,11780017
7,7,Tennessee,6975218
8,8,Texas,29527941
9,9,Virginia,8642274


## Edades

In [35]:
poblacion_edades = pd.read_csv("data_censo/edades.csv")

In [36]:
edades = pd.DataFrame(poblacion_edades.iloc[2:20,0:])
edades.iloc[:,-10:] = edades.iloc[:,-10:].apply(lambda x: x.str.replace(',', '').astype(int))

In [37]:
# Ampliando rango de edades
rango_aum = edades[3:4]
rango_aum["Label (Grouping)"] = "15 to 17 years"
rango_aum2 = rango_aum.iloc[:,-10:] * 0.65
rango_aum2 = rango_aum2.astype(int)
rango_aum.iloc[:,-10:] = rango_aum2
edades = pd.concat([edades, rango_aum])

rango_m = rango_aum.iloc[:,-10:] * 0.35
rango_m = rango_m.astype(int)
rango_aum.iloc[:,-10:] = rango_m
rango_aum["Label (Grouping)"] = "18 to 19 years"
edades[3:4]= rango_aum

In [38]:
edades = pd.melt(edades, id_vars=['Label (Grouping)'], value_vars=edades.columns.to_list()[1:], var_name='estados', value_name='cantidad')
edades["estados"] = edades["estados"].apply(nombre_estados)
edades.columns = ["edad","estados","cantidad"]
edades = pd.merge(edades, estados, how="left" ,on=["estados"])
edades = edades[["index","edad","cantidad"]].reset_index()
edades.columns = ["id","estado","edad","cantidad"]
edades["id"] = edades["id"].astype(str) + edades["estado"].astype(str)
edades["edad"] = edades["edad"].apply(limpiar_edad)
edades.head(3)

,id,estado,edad,cantidad
0,00,0,Under 5 years,402255
1,10,0,5 to 9 years,446309
2,20,0,10 to 14 years,476245


## Etnias

In [139]:
# poblacion_etnias_white = pd.read_csv("data_censo/blancos.csv")
# poblacion_etnias_black = pd.read_csv("data_censo/black.csv")
# poblacion_etnias_american = pd.read_csv("data_censo/etnia_american.csv")
# poblacion_etnias_asian = pd.read_csv("data_censo/etnia_asian.csv")

In [140]:
# etnias_white = etnia(poblacion_etnias_white, "WHITE ALONE")
# etnias_black = etnia(poblacion_etnias_black, "BLACK OR AFRICAN AMERICAN ALONE")
# etnias_american = etnia(poblacion_etnias_american, "AMERICAN INDIAN AND ALASKA NATIVE ALONE")
# etnias_asian = etnia(poblacion_etnias_asian, "ASIAN ALONE")

In [141]:
# tablas_etnias = [etnias_white, etnias_black, etnias_american, etnias_asian]
# tablas_pop_etnias = pd.concat(tablas_etnias)
# tablas_pop_etnias = tablas_pop_etnias.reset_index().reset_index()
# tablas_pop_etnias = tablas_pop_etnias[["level_0", "name", "etiqueta", "nombre", "total"]]
# tablas_pop_etnias['name'] = tablas_pop_etnias['name'].replace('80 to 44 years', '80 to 84 years')

In [142]:
# tablas_pop_etnias = ids_merge(tablas_pop_etnias, estados, edades)

In [143]:
# tablas_pop_etnias = tablas_pop_etnias[["level_0", "id", "nombre", "total"]]
# tablas_pop_etnias.columns = ["id", "edad_id", "etnia", "cantidad_etnias"]
# tablas_pop_etnias.head(3)

## Estado Civil

In [53]:
civil = pd.read_csv("data_censo/estado_civil.csv")

In [54]:
civil = civil.rename(columns=name_columns)
civil.iloc[:,-10:] = civil.iloc[:,-10:].apply(lambda x: x.str.replace(',', '').astype(int))
civil["name"] = civil["name"].apply(limpiar_edad)
civil['name'] = civil['name'].replace('18 and 19 years', '18 to 19 years')

In [55]:
male_never = civil[3:17]
male_married = civil[19:33]
male_separated = civil[35:49]
male_union = civil[50:64]
male_widowed = civil[65:79]
male_divorced = civil[80:94]

female_never = civil[96:110]
female_married = civil[112:126]
female_separated = civil[128:142]
female_union = civil[143:157]
female_widowed = civil[158:172]
female_divorced = civil[173:]

In [56]:
tablas_civil = [male_never, male_married, male_separated, male_union, male_widowed, male_divorced,
                female_never, female_married, female_separated, female_union, female_widowed, female_divorced]

for sex in range(len(tablas_civil)):
    tablas_civil[sex] = rangos(tablas_civil[sex], 11, 0.65, "65 to 69 years", "70 to 74 years")
    tablas_civil[sex] = rangos(tablas_civil[sex], 12, 0.55, "75 to 79 years", "80 to 84 years")

In [57]:
names = ["Soltero", "Casado", "Separado", "Union", "Viudo", "Divorciado"]
data = [(tablas_civil[0], tablas_civil[6]), (tablas_civil[1], tablas_civil[7]), (tablas_civil[2], tablas_civil[8]),
        (tablas_civil[3], tablas_civil[9]), (tablas_civil[4], tablas_civil[10]), (tablas_civil[5], tablas_civil[11])]

estado_civil = pd.DataFrame()

for name, d in zip(names, data):
    df = general(name, d[0], d[1])
    estado_civil = pd.concat([estado_civil, df])

In [58]:
estado_civil = estado_civil.reset_index().reset_index()
estado_civil = estado_civil[["level_0", "name", "etiqueta", "nombre", "total"]]
tabla_estado_civil = ids_merge(estado_civil, estados, edades)
tabla_estado_civil = tabla_estado_civil[["level_0", "id", "nombre", "total"]]
tabla_estado_civil.columns = ["id", "edad_id", "estado_civil", "cantidad_civil"]
tabla_estado_civil.head(3)

,id,edad_id,estado_civil,cantidad_civil
0,0,180,Soltero,286865
1,1,30,Soltero,189569
2,2,40,Soltero,431735


## Tamaño Hogar

In [69]:
hogar = pd.read_csv("data_censo/tam_hogar.csv")

In [70]:
hogar = hogar.rename(columns=name_columns)
hogar.iloc[:,-10:] = hogar.iloc[:,-10:].apply(lambda x: x.str.replace(',', '').astype(int))
one = hogar[3:7]
two = hogar[8:12]

one_r = hogar[14:18]
two_r = hogar[19:]

In [71]:
one = persona(one)
one_r = persona(one_r)
two = persona(two)
two_r = persona(one_r)

In [72]:
one = general("1 persona", one, one_r)
two = general("2 o mas personas", two, two_r)
tam_hogar = pd.concat([one, two])
tam_hogar = tam_hogar.reset_index().reset_index()
tam_hogar = tam_hogar[["level_0", "name", "etiqueta", "nombre", "total"]]
tabla_tam_hogar = ids_merge(tam_hogar, estados, edades)
tabla_tam_hogar = tabla_tam_hogar[["level_0", "id", "nombre", "total"]]
tabla_tam_hogar.columns = ["id", "edad_id", "personas", "cantidad_hogares"]
tabla_tam_hogar.head(3)

,id,edad_id,personas,cantidad_hogares
0,0,180,1 persona,287663
1,1,30,1 persona,287663
2,2,40,1 persona,287663


## Nivel de educación

In [144]:
educacion1 = pd.read_csv("data_censo/educacion1.csv")
educacion2 = pd.read_csv("data_censo/educacion2.csv")

In [145]:
educacion1 = educacion1.rename(columns=name_columns)
educacion2 = educacion2.rename(columns=name_columns)

In [146]:
educacion1.iloc[:,-10:] = educacion1.iloc[:,-10:].apply(lambda x: x.str.replace(',', '').astype(int))
educacion2.iloc[:,-10:] = educacion2.iloc[:,-10:].apply(lambda x: x.str.replace(',', '').astype(int))

In [147]:
educacion1["name"] = educacion1["name"].apply(limpiar_edad)
educacion2["name"] = educacion2["name"].apply(limpiar_edad)

In [148]:
male_public = educacion1[3:11]
male_private = educacion1[12:20]
male_not_enrolled = educacion1[21:29]

female_public = educacion1[31:39]
female_private = educacion1[40:48]
female_not_enrolled = educacion1[49:]

In [149]:
publica = pd.merge(male_public, female_public, on="name")
privada = pd.merge(male_private, female_private, on="name")
not_enrolled = pd.merge(male_not_enrolled, female_not_enrolled, on="name")

In [150]:
publica = uniones(publica, "Inscrito")
privada = uniones(privada, "Inscrito")
not_enrolled = uniones(not_enrolled[1:4], "No inscrito")

In [151]:
inscritos = pd.merge(publica[1:4].drop(['nombre'], axis=1), privada[1:4].drop(['nombre'], axis=1), on="name")

In [152]:
inscritos = inscritos.rename(columns={
    'Arizona_x_x': 'Arizona_x', 'Florida_x_x':'Florida_x', 'Georgia_x_x':'Georgia_x', 'Illinois_x_x':'Illinois_x',
    'Michigan_x_x':'Michigan_x', 'North Carolina_x_x':'North Carolina_x', 'Ohio_x_x':'Ohio_x',
    'Tennessee_x_x':'Tennessee_x', 'Texas_x_x':'Texas_x', 'Virginia_x_x':'Virginia_x',
    'Arizona_x_y':'Arizona_y', 'Florida_x_y':'Florida_y', 'Georgia_x_y':'Georgia_y', 'Illinois_x_y':'Illinois_y', 
    'Michigan_x_y':'Michigan_y', 'North Carolina_x_y':'North Carolina_y', 'Ohio_x_y':'Ohio_y', 'Tennessee_x_y':'Tennessee_y',
    'Texas_x_y':'Texas_y', 'Virginia_x_y':'Virginia_y'
})

In [153]:
inscritos = uniones(inscritos, "Inscritos")

rangos = ["5 to 9 years", "10 to 14 years", "15 to 17 years"]
inscritos.nombre = rangos
inscritos.name = ["Less than 9th grade", "Less than 9th grade", "9th to 12th grade, no diploma"]

not_enrolled.nombre = rangos
not_enrolled.name = ["Not enrolled", "Not enrolled", "High school graduate (includes equivalency)"]

In [154]:
inscritos = princ(inscritos)
not_enrolled = princ(not_enrolled)

In [155]:
male_18 = educacion2[3:10]
male_25 = educacion2[11:18]
male_35 = educacion2[19:26]
male_45 = educacion2[27:34]
male_65 = educacion2[35:42]

female_18 = educacion2[44:51]
female_25 = educacion2[52:59]
female_35 = educacion2[60:67]
female_45 = educacion2[68:75]
female_65 = educacion2[76:]

In [156]:
rango_18 = general("18 to 19 years", male_18, female_18) 
rango_25 = general("25 to 29 years", male_25, female_25) 
rango_35 = general("35 to 39 years", male_35, female_35)
rango_45 = general("45 to 49 years", male_45, female_45) 
rango_65 = general("65 to 69 years", male_65, female_65)

rango_20 = ultimos(rango_18, 0.65).assign(nombre="20 to 24 years")
rango_30 = ultimos(rango_25, 0.45).assign(nombre="30 to 34 years")
rango_40 = ultimos(rango_35, 0.45).assign(nombre="40 to 44 years")
rango_50 = ultimos(rango_45, 0.35).assign(nombre="50 to 54 years")
rango_55 = ultimos(rango_45, 0.20).assign(nombre="55 to 59 years")
rango_60 = ultimos(rango_45, 0.20).assign(nombre="60 to 64 years")
rango_70 = ultimos(rango_65, 0.21).assign(nombre="70 to 74 years")
rango_75 = ultimos(rango_65, 0.19).assign(nombre="75 to 79 years")
rango_80 = ultimos(rango_65, 0.17).assign(nombre="80 to 84 years")
rango_85 = ultimos(rango_65, 0.13).assign(nombre="85 years and over")

rango_18 = ultimos(rango_18, 0.45)
rango_25 = ultimos(rango_25, 0.65)
rango_35 = ultimos(rango_35, 0.65)
rango_45 = ultimos(rango_45, 0.45)
rango_65 = ultimos(rango_65, 0.30)

rangos_educacion = [inscritos, not_enrolled, rango_18, rango_20, rango_25, rango_30, rango_35, rango_40, rango_45, rango_50, rango_55, rango_60, rango_65, rango_70, rango_75, rango_80, rango_85]
nivel_educacion = pd.concat(rangos_educacion)


In [157]:
nivel_educacion = nivel_educacion.reset_index().reset_index()
nivel_educacion = nivel_educacion[["level_0", "name", "etiqueta", "nombre", "total"]]

In [158]:
nivel_educacion = nivel_educacion.rename(columns={"name": "nombre", "nombre":"name"})

In [159]:
tabla_nivel_educacion = ids_merge(nivel_educacion, estados, edades)

In [160]:
tabla_nivel_educacion = tabla_nivel_educacion[["level_0", "id", "nombre", "total"]]
tabla_nivel_educacion.columns = ["id", "edad_id", "grado", "cantidad_educacion"]
tabla_nivel_educacion.head(3)

,id,edad_id,grado,cantidad_educacion
0,0,10,Less than 9th grade,412103
1,1,20,Less than 9th grade,465369
2,2,180,"9th to 12th grade, no diploma",275584


## Ingresos

In [135]:
poblacion_ingresos = pd.read_csv("data_censo/ingresos.csv")

In [136]:
ingresos = poblacion_ingresos.iloc[1:,:]
ingresos.iloc[:,-10:] = ingresos.iloc[:,-10:].apply(lambda x: x.str.replace(',', '').astype(int))
ingresos["Label (Grouping)"] = ingresos["Label (Grouping)"].apply(limpiar_edad)
edades_ingresos = ingresos["Label (Grouping)"][ingresos["Label (Grouping)"].str.contains(r'(.*Householder.*)')].tolist()
rango_edades = [edades_ingresos[0]]*16 + [edades_ingresos[1]]*16 + [edades_ingresos[2]]*16 + [edades_ingresos[3]]*16
ingresos = ingresos.drop([1,18,35,52])
ingresos["edades"] = rango_edades
ingresos = pd.melt(ingresos, id_vars=['Label (Grouping)', 'edades'], value_vars=ingresos.columns.to_list()[1:-1], var_name='estado', value_name='cantidad')
ingresos.columns = ["ingresos","edad","estado","cantidad"]
ingresos["estado"] = ingresos["estado"].apply(nombre_estados)

In [137]:
new_rows = []
for i, row in ingresos.iterrows():
    age_list = age_range_to_list(row["edad"])
    for age in age_list:
        new_row = row.copy()
        if(type(age) is int):
            if(age == 5):
                new_row["edad"] = f"Under {age} years"
            elif(age == 85):
                new_row["edad"] = f"{age} years and over"
        else:
            new_row["edad"] = f"{age[0]} to {age[1]} years"
        new_rows.append(new_row)
tabla_ingresos = pd.DataFrame(new_rows)
tabla_ingresos = pd.merge(tabla_ingresos, estados, how="left" ,left_on='estado', right_on='estados')
tabla_ingresos = tabla_ingresos.rename(columns={"estado":"estado2",
                                  "index":"estado"})
tabla_ingresos = pd.merge(tabla_ingresos, edades, how="left", on=["edad","estado"]).reset_index()
tabla_ingresos = tabla_ingresos[["index","id","ingresos","cantidad_x"]]
tabla_ingresos.columns = ["id","edad_id","ingresos","cantidad_ingresos"]
tabla_ingresos.head(3)

,id,edad_id,ingresos,cantidad_ingresos
0,0,00,"Less than $10,000",13603
1,1,10,"Less than $10,000",13603
2,2,20,"Less than $10,000",13603


## CENSO UNIFICADO

In [201]:
censo = pd.merge(estados, edades, how="left", left_on="index", right_on="estado", suffixes=("_Estados","_Edades"))
censo = pd.merge(censo, tabla_nivel_educacion, how="inner", left_on="id", right_on="edad_id", suffixes=("","_Educacion"))
censo = pd.merge(censo, tabla_estado_civil, how="inner", left_on="id", right_on="edad_id", suffixes=("","_Civil"))
censo = pd.merge(censo, tabla_tam_hogar, how="left", left_on="id", right_on="edad_id", suffixes=("","_Hogar"))
censo = pd.merge(censo, tabla_ingresos, how="left", left_on="id", right_on="edad_id", suffixes=("","_Ingreso"))
censo = censo[['estados', 'poblacion',
               'edad', 'cantidad', 
               'grado', 'cantidad_educacion',
               'estado_civil', 'cantidad_civil',
               'personas', 'cantidad_hogares',
               'ingresos', 'cantidad_ingresos']]
censo

,estados,poblacion,edad,cantidad,grado,cantidad_educacion,estado_civil,cantidad_civil,personas,cantidad_hogares,ingresos,cantidad_ingresos
0,Arizona,7276316,18 to 19 years,109881,Less than 9th grade,3691,Soltero,189569,1 persona,287663,"Less than $10,000",13603
1,Arizona,7276316,18 to 19 years,109881,Less than 9th grade,3691,Soltero,189569,1 persona,287663,"$10,000 to $14,999",6050
2,Arizona,7276316,18 to 19 years,109881,Less than 9th grade,3691,Soltero,189569,1 persona,287663,"$15,000 to $19,999",5631
3,Arizona,7276316,18 to 19 years,109881,Less than 9th grade,3691,Soltero,189569,1 persona,287663,"$20,000 to $24,999",9033
4,Arizona,7276316,18 to 19 years,109881,Less than 9th grade,3691,Soltero,189569,1 persona,287663,"$25,000 to $29,999",8104
...,...,...,...,...,...,...,...,...,...,...,...,...
205435,Virginia,8642274,15 to 17 years,371036,High school graduate (includes equivalency),9415,Divorciado,206,2 o mas personas,1154204,"$75,000 to $99,999",10337
205436,Virginia,8642274,15 to 17 years,371036,High school graduate (includes equivalency),9415,Divorciado,206,2 o mas personas,1154204,"$100,000 to $124,999",5873
205437,Virginia,8642274,15 to 17 years,371036,High school graduate (includes equivalency),9415,Divorciado,206,2 o mas personas,1154204,"$125,000 to $149,999",2406
205438,Virginia,8642274,15 to 17 years,371036,High school graduate (includes equivalency),9415,Divorciado,206,2 o mas personas,1154204,"$150,000 to $199,999",2900


# Perfiles

In [285]:
perfiles_edades = censo[["estados","edad","cantidad"]].drop_duplicates()
perfiles_edades['Porcentaje'] = (perfiles_edades['cantidad'] / perfiles_edades.groupby('estados')['cantidad'].transform('sum')).astype(float).round(3)
perfiles_edades['Rango'] = perfiles_edades.groupby('estados')['Porcentaje'].rank(ascending=False, method='dense')
perfiles_edades = perfiles_edades[perfiles_edades['Rango'].eq(1) | perfiles_edades.duplicated(['estados', 'Porcentaje'], keep=False)]
perfiles_edades = perfiles_edades[perfiles_edades["Rango"] == 1]
perfiles_edades = perfiles_edades.drop('Rango', axis=1)
perfiles_edades

,estados,edad,cantidad,Porcentaje
2688,Arizona,25 to 29 years,502697,0.085
32640,Florida,60 to 64 years,1509806,0.083
45120,Georgia,30 to 34 years,738064,0.085
46464,Georgia,35 to 39 years,733823,0.085
65664,Illinois,30 to 34 years,858792,0.084
67008,Illinois,35 to 39 years,864721,0.084
94272,Michigan,60 to 64 years,723695,0.088
104064,North Carolina,20 to 24 years,705531,0.082
106752,North Carolina,30 to 34 years,702577,0.082
112128,North Carolina,50 to 54 years,698866,0.082


## Perfil Arizona

In [314]:
perfil_arizona = censo[censo["estados"] == "Arizona"]
perfil_arizona = perfil_arizona[perfil_arizona["edad"] == "25 to 29 years"]
perfil_arizona["total"] = perfil_arizona["cantidad"].astype(int) + perfil_arizona["cantidad_educacion"].astype(int) + perfil_arizona["cantidad_civil"].astype(int) + perfil_arizona["cantidad_hogares"].astype(int) + perfil_arizona["cantidad_ingresos"].astype(int)
perfil_arizona['porcentaje'] = perfil_arizona['total'] /perfil_arizona['total'].sum()
perfil_arizona['rango'] = perfil_arizona['porcentaje'].rank(ascending=False, method='dense')
perfil_arizona = perfil_arizona[perfil_arizona['rango'].eq(1) | perfil_arizona.duplicated(['estados', 'porcentaje'], keep=False)]
perfil_arizona = perfil_arizona[perfil_arizona["rango"] == 1]
perfil_arizona = perfil_arizona.drop(['rango','porcentaje'], axis=1)
perfil_arizona

,estados,poblacion,edad,cantidad,grado,cantidad_educacion,estado_civil,cantidad_civil,personas,cantidad_hogares,ingresos,cantidad_ingresos,total
3099,Arizona,7276316,25 to 29 years,502697,High school graduate (includes equivalency),73875,Soltero,324093,2 o mas personas,912290,"$75,000 to $99,999",145275,1958230


## Perfil Florida

In [315]:
perfil_florida = censo[censo["estados"] == "Florida"]
perfil_florida = perfil_florida[perfil_florida["edad"] == "60 to 64 years"]
perfil_florida["total"] = perfil_florida["cantidad"].astype(int) + perfil_florida["cantidad_educacion"].astype(int) + perfil_florida["cantidad_civil"].astype(int) + perfil_florida["cantidad_hogares"].astype(int) + perfil_florida["cantidad_ingresos"].astype(int)
perfil_florida['porcentaje'] = perfil_florida['total'] /perfil_florida['total'].sum()
perfil_florida['rango'] = perfil_florida['porcentaje'].rank(ascending=False, method='dense')
perfil_florida = perfil_florida[perfil_florida['rango'].eq(1) | perfil_florida.duplicated(['estados', 'porcentaje'], keep=False)]
perfil_florida = perfil_florida[perfil_florida["rango"] == 1]
perfil_florida = perfil_florida.drop(['rango','porcentaje'], axis=1)
perfil_florida

,estados,poblacion,edad,cantidad,grado,cantidad_educacion,estado_civil,cantidad_civil,personas,cantidad_hogares,ingresos,cantidad_ingresos,total
33083,Florida,21781128,60 to 64 years,1509806,High school graduate (includes equivalency),22404,Casado,864393,2 o mas personas,1509617,"$75,000 to $99,999",421462,4327682


## Perfil Georgia

In [341]:
perfil_georgia = censo[censo["estados"] == "Georgia"]
perfil_georgia = perfil_georgia[(perfil_georgia["edad"] == "30 to 34 years") | (perfil_georgia["edad"] == "35 to 39 years")]
perfil_georgia["total"] = perfil_georgia["cantidad"].astype(int) + perfil_georgia["cantidad_educacion"].astype(int) + perfil_georgia["cantidad_civil"].astype(int) + perfil_georgia["cantidad_hogares"].astype(int) + perfil_georgia["cantidad_ingresos"].astype(int)
perfil_georgia['porcentaje'] = (perfil_georgia['total'] /perfil_georgia.groupby('edad')['total'].transform('sum')).astype(float).round(6)
perfil_georgia['rango'] = perfil_georgia.groupby('edad')['porcentaje'].rank(ascending=False, method='dense')
perfil_georgia = perfil_georgia[perfil_georgia['rango'].eq(1) | perfil_georgia.duplicated(['edad', 'porcentaje'], keep=False)]
perfil_georgia = perfil_georgia[perfil_georgia["rango"] == 1]
perfil_georgia = perfil_georgia.drop(['rango','porcentaje'], axis=1)
perfil_georgia

,estados,poblacion,edad,cantidad,grado,cantidad_educacion,estado_civil,cantidad_civil,personas,cantidad_hogares,ingresos,cantidad_ingresos,total
45531,Georgia,10799566,30 to 34 years,738064,High school graduate (includes equivalency),167755,Soltero,334186,2 o mas personas,1403391,"$75,000 to $99,999",194670,2838066
46107,Georgia,10799566,30 to 34 years,738064,Bachelor's degree,167832,Soltero,334186,2 o mas personas,1403391,"$75,000 to $99,999",194670,2838143
46907,Georgia,10799566,35 to 39 years,733823,High school graduate (includes equivalency),96896,Casado,398585,2 o mas personas,1403391,"$75,000 to $99,999",194670,2827365


## Perfil Illinois

In [351]:
perfil_illinois = censo[censo["estados"] == "Illinois"]
perfil_illinois = perfil_illinois[(perfil_illinois["edad"] == "30 to 34 years") | (perfil_illinois["edad"] == "35 to 39 years")]
perfil_illinois["total"] = perfil_illinois["cantidad"].astype(int) + perfil_illinois["cantidad_educacion"].astype(int) + perfil_illinois["cantidad_civil"].astype(int) + perfil_illinois["cantidad_hogares"].astype(int) + perfil_illinois["cantidad_ingresos"].astype(int)
perfil_illinois['porcentaje'] = (perfil_illinois['total'] /perfil_illinois.groupby('edad')['total'].transform('sum')).astype(float).round(6)
perfil_illinois['rango'] = perfil_illinois.groupby('edad')['porcentaje'].rank(ascending=False, method='dense')
perfil_illinois = perfil_illinois[perfil_illinois['rango'].eq(1) | perfil_illinois.duplicated(['edad', 'porcentaje'], keep=False)]
perfil_illinois = perfil_illinois[perfil_illinois["rango"] == 1]
perfil_illinois = perfil_illinois.drop(['rango','porcentaje'], axis=1)
perfil_illinois

,estados,poblacion,edad,cantidad,grado,cantidad_educacion,estado_civil,cantidad_civil,personas,cantidad_hogares,ingresos,cantidad_ingresos,total
66651,Illinois,12671469,30 to 34 years,858792,Bachelor's degree,228681,Soltero,408351,2 o mas personas,1753273,"$75,000 to $99,999",246279,3495376
68027,Illinois,12671469,35 to 39 years,864721,Bachelor's degree,123038,Casado,496845,2 o mas personas,1753273,"$75,000 to $99,999",246279,3484156


## Perfil Michigan

In [363]:
perfil_michigan = censo[censo["estados"] == "Michigan"]
perfil_michigan = perfil_michigan[perfil_michigan["edad"] == "60 to 64 years"]
perfil_michigan["total"] = perfil_michigan["cantidad"].astype(int) + perfil_michigan["cantidad_educacion"].astype(int) + perfil_michigan["cantidad_civil"].astype(int) + perfil_michigan["cantidad_hogares"].astype(int) + perfil_michigan["cantidad_ingresos"].astype(int)
perfil_michigan['porcentaje'] = (perfil_michigan['total'] /perfil_michigan['total'].sum()).astype(float).round(6)
perfil_michigan['rango'] = perfil_michigan['porcentaje'].rank(ascending=False, method='dense')
perfil_michigan = perfil_michigan[perfil_michigan['rango'].eq(1) | perfil_michigan.duplicated(['estados', 'porcentaje'], keep=False)]
perfil_michigan = perfil_michigan[perfil_michigan["rango"] == 1]
perfil_michigan = perfil_michigan.drop(['rango','porcentaje'], axis=1)
perfil_michigan

,estados,poblacion,edad,cantidad,grado,cantidad_educacion,estado_civil,cantidad_civil,personas,cantidad_hogares,ingresos,cantidad_ingresos,total
94715,Michigan,10050811,60 to 64 years,723695,High school graduate (includes equivalency),10621,Casado,443712,2 o mas personas,751468,"$75,000 to $99,999",197063,2126559


## Perfil North Carolina

In [371]:
perfil_carolina = censo[censo["estados"] == "North Carolina"]
perfil_carolina = perfil_carolina[(perfil_carolina["edad"] == "20 to 24 years") | (perfil_carolina["edad"] == "30 to 34 years") | (perfil_carolina["edad"] == "50 to 54 years")]
perfil_carolina["total"] = perfil_carolina["cantidad"].astype(int) + perfil_carolina["cantidad_educacion"].astype(int) + perfil_carolina["cantidad_civil"].astype(int) + perfil_carolina["cantidad_hogares"].astype(int) + perfil_carolina["cantidad_ingresos"].astype(int)
perfil_carolina['porcentaje'] = (perfil_carolina['total'] /perfil_carolina.groupby('edad')['total'].transform('sum')).astype(float).round(6)
perfil_carolina['rango'] = perfil_carolina.groupby('edad')['porcentaje'].rank(ascending=False, method='dense')
perfil_carolina = perfil_carolina[perfil_carolina['rango'].eq(1) | perfil_carolina.duplicated(['edad', 'porcentaje'], keep=False)]
perfil_carolina = perfil_carolina[perfil_carolina["rango"] == 1]
perfil_carolina = perfil_carolina.drop(['rango','porcentaje'], axis=1)
perfil_carolina

,estados,poblacion,edad,cantidad,grado,cantidad_educacion,estado_civil,cantidad_civil,personas,cantidad_hogares,ingresos,cantidad_ingresos,total
104656,North Carolina,10551162,20 to 24 years,705531,"Some college, no degree",238635,Soltero,616977,2 o mas personas,1406562,"Less than $10,000",31793,2999498
107771,North Carolina,10551162,30 to 34 years,702577,Bachelor's degree,170025,Casado,318210,2 o mas personas,1406562,"$75,000 to $99,999",192070,2789444
112571,North Carolina,10551162,50 to 54 years,698866,High school graduate (includes equivalency),236574,Casado,428246,2 o mas personas,1406562,"$75,000 to $99,999",203699,2973947


## Perfil Ohio

In [376]:
perfil_ohio = censo[censo["estados"] == "Ohio"]
perfil_ohio = perfil_ohio[perfil_ohio["edad"] == "60 to 64 years"]
perfil_ohio["total"] = perfil_ohio["cantidad"].astype(int) + perfil_ohio["cantidad_educacion"].astype(int) + perfil_ohio["cantidad_civil"].astype(int) + perfil_ohio["cantidad_hogares"].astype(int) + perfil_ohio["cantidad_ingresos"].astype(int)
perfil_ohio['porcentaje'] = (perfil_ohio['total'] /perfil_ohio['total'].sum()).astype(float).round(6)
perfil_ohio['rango'] = perfil_ohio['porcentaje'].rank(ascending=False, method='dense')
perfil_ohio = perfil_ohio[perfil_ohio['rango'].eq(1) | perfil_ohio.duplicated(['estados', 'porcentaje'], keep=False)]
perfil_ohio = perfil_ohio[perfil_ohio["rango"] == 1]
perfil_ohio = perfil_ohio.drop(['rango','porcentaje'], axis=1)
perfil_ohio

,estados,poblacion,edad,cantidad,grado,cantidad_educacion,estado_civil,cantidad_civil,personas,cantidad_hogares,ingresos,cantidad_ingresos,total
135803,Ohio,11780017,60 to 64 years,823595,High school graduate (includes equivalency),14196,Casado,488219,2 o mas personas,902380,"$75,000 to $99,999",243810,2472200


## Perfil Tennessee

In [378]:
perfil_tennessee = censo[censo["estados"] == "Tennessee"]
perfil_tennessee = perfil_tennessee[perfil_tennessee["edad"] == "25 to 29 years"]
perfil_tennessee["total"] = perfil_tennessee["cantidad"].astype(int) + perfil_tennessee["cantidad_educacion"].astype(int) + perfil_tennessee["cantidad_civil"].astype(int) + perfil_tennessee["cantidad_hogares"].astype(int) + perfil_tennessee["cantidad_ingresos"].astype(int)
perfil_tennessee['porcentaje'] = (perfil_tennessee['total'] /perfil_tennessee['total'].sum()).astype(float).round(6)
perfil_tennessee['rango'] = perfil_tennessee['porcentaje'].rank(ascending=False, method='dense')
perfil_tennessee = perfil_tennessee[perfil_tennessee['rango'].eq(1) | perfil_tennessee.duplicated(['estados', 'porcentaje'], keep=False)]
perfil_tennessee = perfil_tennessee[perfil_tennessee["rango"] == 1]
perfil_tennessee = perfil_tennessee.drop(['rango','porcentaje'], axis=1)
perfil_tennessee

,estados,poblacion,edad,cantidad,grado,cantidad_educacion,estado_civil,cantidad_civil,personas,cantidad_hogares,ingresos,cantidad_ingresos,total
146907,Tennessee,6975218,25 to 29 years,474393,High school graduate (includes equivalency),78697,Soltero,279968,2 o mas personas,920025,"$75,000 to $99,999",139346,1892429


## Perfil Texas

In [381]:
perfil_texas = censo[censo["estados"] == "Texas"]
perfil_texas = perfil_texas[perfil_texas["edad"] == "30 to 34 years"]
perfil_texas["total"] = perfil_texas["cantidad"].astype(int) + perfil_texas["cantidad_educacion"].astype(int) + perfil_texas["cantidad_civil"].astype(int) + perfil_texas["cantidad_hogares"].astype(int) + perfil_texas["cantidad_ingresos"].astype(int)
perfil_texas['porcentaje'] = (perfil_texas['total'] /perfil_texas['total'].sum()).astype(float).round(6)
perfil_texas['rango'] = perfil_texas['porcentaje'].rank(ascending=False, method='dense')
perfil_texas = perfil_texas[perfil_texas['rango'].eq(1) | perfil_texas.duplicated(['estados', 'porcentaje'], keep=False)]
perfil_texas = perfil_texas[perfil_texas["rango"] == 1]
perfil_texas = perfil_texas.drop(['rango','porcentaje'], axis=1)
perfil_texas

,estados,poblacion,edad,cantidad,grado,cantidad_educacion,estado_civil,cantidad_civil,personas,cantidad_hogares,ingresos,cantidad_ingresos,total
168827,Texas,29527941,30 to 34 years,2158231,High school graduate (includes equivalency),479449,Casado,1055917,2 o mas personas,4045349,"$75,000 to $99,999",550728,8289674
169403,Texas,29527941,30 to 34 years,2158231,Bachelor's degree,481430,Casado,1055917,2 o mas personas,4045349,"$75,000 to $99,999",550728,8291655


## Perfil Virginia

In [384]:
perfil_virginia = censo[censo["estados"] == "Virginia"]
perfil_virginia = perfil_virginia[perfil_virginia["edad"] == "35 to 39 years"]
perfil_virginia["total"] = perfil_virginia["cantidad"].astype(int) + perfil_virginia["cantidad_educacion"].astype(int) + perfil_virginia["cantidad_civil"].astype(int) + perfil_virginia["cantidad_hogares"].astype(int) + perfil_virginia["cantidad_ingresos"].astype(int)
perfil_virginia['porcentaje'] = (perfil_virginia['total'] /perfil_virginia['total'].sum()).astype(float).round(6)
perfil_virginia['rango'] = perfil_virginia['porcentaje'].rank(ascending=False, method='dense')
perfil_virginia = perfil_virginia[perfil_virginia['rango'].eq(1) | perfil_virginia.duplicated(['estados', 'porcentaje'], keep=False)]
perfil_virginia = perfil_virginia[perfil_virginia["rango"] == 1]
perfil_virginia = perfil_virginia.drop(['rango','porcentaje'], axis=1)
perfil_virginia

,estados,poblacion,edad,cantidad,grado,cantidad_educacion,estado_civil,cantidad_civil,personas,cantidad_hogares,ingresos,cantidad_ingresos,total
191291,Virginia,8642274,35 to 39 years,600446,Bachelor's degree,86465,Casado,345506,2 o mas personas,1154204,"$75,000 to $99,999",155625,2342246


## Perfiles unificados

In [391]:
perfiles.columns

Index(['estados', 'poblacion', 'edad', 'cantidad', 'grado',
       'cantidad_educacion', 'estado_civil', 'cantidad_civil', 'personas',
       'cantidad_hogares', 'ingresos', 'cantidad_ingresos', 'total'],
      dtype='object')

In [392]:
perfiles = pd.concat([perfil_arizona,
           perfil_florida,
           perfil_georgia,
           perfil_illinois, 
           perfil_michigan, 
           perfil_carolina, 
           perfil_ohio, 
           perfil_tennessee, 
           perfil_texas, 
           perfil_virginia])
perfiles = perfiles[['estados', 'edad', 'grado', 'estado_civil', 'personas', 'ingresos']]
perfiles

,estados,edad,grado,estado_civil,personas,ingresos
3099,Arizona,25 to 29 years,High school graduate (includes equivalency),Soltero,2 o mas personas,"$75,000 to $99,999"
33083,Florida,60 to 64 years,High school graduate (includes equivalency),Casado,2 o mas personas,"$75,000 to $99,999"
45531,Georgia,30 to 34 years,High school graduate (includes equivalency),Soltero,2 o mas personas,"$75,000 to $99,999"
46107,Georgia,30 to 34 years,Bachelor's degree,Soltero,2 o mas personas,"$75,000 to $99,999"
46907,Georgia,35 to 39 years,High school graduate (includes equivalency),Casado,2 o mas personas,"$75,000 to $99,999"
66651,Illinois,30 to 34 years,Bachelor's degree,Soltero,2 o mas personas,"$75,000 to $99,999"
68027,Illinois,35 to 39 years,Bachelor's degree,Casado,2 o mas personas,"$75,000 to $99,999"
94715,Michigan,60 to 64 years,High school graduate (includes equivalency),Casado,2 o mas personas,"$75,000 to $99,999"
104656,North Carolina,20 to 24 years,"Some college, no degree",Soltero,2 o mas personas,"Less than $10,000"
107771,North Carolina,30 to 34 years,Bachelor's degree,Casado,2 o mas personas,"$75,000 to $99,999"
